# 5.2 RAG 기반의 챗봇 만들기
- 라이브러리: langchain, openai, unstructured sentence-transfomers, chromadb
- 언어모델: gpt-3.5-turbo
- 임베딩모델: all_MiniLM-L6-v2
- 벡터 데이터베이스: 크로마(Chroma)

In [1]:
# 텍스트 파일 같은 구조화 되지 않은 데이터를 다루는데 사용
# !pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 16.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.2 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=6143ac2e08e3ef09ed66e91b1f556dfa9f919a074a6ecf318f208a08b9f5a55e
  Stored in directory: /Users/kimkyusan/Library/Caches/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
  Created wheel for lxml: filename=lxml-5.3.0-cp38-cp38-macosx_11_0_arm64.whl size=1484275 sha256=2fd5144e21cffc8a643a9743ca3d6f39cd193c81d

In [2]:
# 문장을 벡터로 변환하고 이를 통해 텍스트 데이터의 의미적 유사성을 계산하기 위해 사용
# !pip install sentence-transformers

In [3]:
# 벡터를 저장하고 유사도 검색을 지원
# !pip install chromadb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 18.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 17.7 MB/s eta 0:00:00a 0:00:01
Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.4 MB/s eta 0:00:00
Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
  Created wheel for pypika: filename=PyPika-0.48.9-py2.py3-none-any.whl size=53726 sha256=5eace0f9d514c90393ad91bc0560d32eb3579812fe95492da5c98ecaadf9dc46
  Stored in directory: /Users/kimkyusan/Li

## 텍스트 파일 불러오기

In [9]:
from langchain.document_loaders import TextLoader
file_path = './data/AI.txt'

documents = TextLoader(file_path).load() # AI.txt 파일 위치 지정

### 문장을 청크(작은 덩어리)로 분할
- RecursiveCharacterTextSplitter


In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 문장을 청크로 분할
def split_docs(documents, chunk_size = 1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size= chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

# docs 변수에 분할 문서를 저장
docs = split_docs(documents)

### 벡터 데이터베이스인 크로마에 임베딩 처리된 벡터를 저장

In [13]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name= 'all-MiniLM-L6-v2')

# Chromdb에 벡터 저장
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)

/var/folders/f7/gl6wpvqx5z596rbpjsq5n3tc0000gn/T/ipykernel_32662/4256478109.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = SentenceTransformerEmbeddings(model_name= 'all-MiniLM-L6-v2')
/opt/anaconda3/envs/llm_project/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/opt/anaconda3/envs/llm_project/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set

### 텍스트 파일에서 관련 내용을 찾아서 LLM 에 제공하면, LLM 답변 생성

In [14]:
import os
# os.environ["OPENAI_API_KEY"] = ""

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
model_name = "gpt-3.5-turbo"

llm = ChatOpenAI(model_name = model_name)

# Q&A 체인을 사용하여 쿼리에 대한 답변 얻기
# from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type = "stuff", verbose= True)

# 쿼리를 작성하고 유사도 검색을 수행하여 답변을 생성, 따라서 택스트에 있는 내용을 질문해야 합니다.
query = "AI 란?"
matching_docs = db.similarity_search(query)
answer = chain.run(input_documents = matching_docs, question=query)
answer

/var/folders/f7/gl6wpvqx5z596rbpjsq5n3tc0000gn/T/ipykernel_32662/2058081468.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model_name = model_name)
/var/folders/f7/gl6wpvqx5z596rbpjsq5n3tc0000gn/T/ipykernel_32662/2058081468.py:9: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chai



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
The various sub-fields of AI research are centered around particular goals and the use of particular tools. The traditional goals of AI research include reasoning, knowledge representation, planning, learning, natural language processing, perception, and support for robotics.[a] General intelligence (the ability to complete any task performable by a human) is among the field's long-term goals.[11]

To solve these problems, AI researchers have adapted and integrated a wide range of problem-solving techniques, including search and mathematical optimization, formal logic, artificial neural networks, and methods based on statistics, operations research, and economics.[b] AI also draws upon psychology, li

'AI(인공지능)란 사람이나 동물의 지능과는 달리 기계나 소프트웨어의 지능을 의미합니다. 컴퓨터 과학 분야에서 연구 및 개발되는 지능적인 기계들을 연구하는 학문 분야입니다. AI 기술은 산업, 정부, 과학 분야에서 광범위하게 활용되고 있습니다. 예를 들어, 고급 웹 검색 엔진(Google Search), 추천 시스템(YouTube, Amazon, Netflix), 인간의 음성 인식(Google Assistant, Siri, Alexa), 자율 주행 자동차(Waymo), 생성적 창작 도구(ChatGPT, AI art), 전략 게임에서 초인간적인 플레이와 분석(체스, 바둑) 등의 분야에 AI 기술이 널리 사용되고 있습니다.'